# Apply filters to Nanoimager csv and write a filtered csv

**ONI Nanoimager is a commercial widefield microscope which is used for localisation microscopy. The Nanoimager software has GUI to filter data which works great. Although, it is difficult to document the filter settings. Here I show how I use python pandas to filter the data and export it as a csv.**

## Load and clean data

### Load data

In [1]:
# Here I import the python packages.
import pandas as pd
import numpy as np

In [2]:
# Load the localisation file from the nanoimager as a dataframe
df = pd.read_csv('Nanoimager_localizations.csv')

In [3]:
# Here I look at the top two rows of the dataframe.
df.head(2)

,Channel,Frame,X (nm),Y (nm),Z (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (pix),PSF Sigma Y (pix)
0,1,18,17393.363281,5341.088867,0.0,12.122814,14.760088,576.440552,45.888653,1.065249,1.301303
1,1,18,28469.585938,6503.035645,0.0,19.043039,8.590194,2415.484863,45.925720,1.979784,1.528099


In [4]:
# Here,I look at the column names, their datatypes and the number of rows.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485058 entries, 0 to 485057
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Channel            485058 non-null  int64  
 1   Frame              485058 non-null  int64  
 2   X (nm)             485058 non-null  float64
 3   Y (nm)             485058 non-null  float64
 4   Z (nm)             485058 non-null  float64
 5   X precision (nm)   485058 non-null  float64
 6   Y precision (nm)   485058 non-null  float64
 7   Photons            485058 non-null  float64
 8   Background         485058 non-null  float64
 9   PSF Sigma X (pix)  485058 non-null  float64
 10  PSF Sigma Y (pix)  485058 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 40.7 MB


### Clean data

Choose the colour channel (green 0 or red 1)

In [5]:
# Here I only want channel 1 or the red channel
df_selected=df[df.Channel == 1] # filter channel 1, use 0 for the green channel
df_selected.shape # In this case we only had channel 1 so no change in number of rows.

(485058, 11)

Drop useless columns

In [6]:
# channel and z columns are useless so I drop them
df_selected.drop(columns=['Channel', 'Z (nm)'], inplace=True)
df_selected.head(2)

,Frame,X (nm),Y (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (pix),PSF Sigma Y (pix)
0,18,17393.363281,5341.088867,12.122814,14.760088,576.440552,45.888653,1.065249,1.301303
1,18,28469.585938,6503.035645,19.043039,8.590194,2415.484863,45.925720,1.979784,1.528099


Fix data

In [7]:
# PSF should be in nm but ONI has left it as pixel
# convert PSF to nm and change the column names
pixel_size=117
df_selected.rename(columns={"PSF Sigma X (pix)": "PSF Sigma X (nm)", "PSF Sigma Y (pix)": "PSF Sigma Y (nm)"}, inplace=True)
df_selected["PSF Sigma X (nm)"]=pixel_size*df_selected["PSF Sigma X (nm)"]
df_selected["PSF Sigma Y (nm)"]=pixel_size*df_selected["PSF Sigma Y (nm)"]
df_selected.head(2)

,Frame,X (nm),Y (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (nm),PSF Sigma Y (nm)
0,18,17393.363281,5341.088867,12.122814,14.760088,576.440552,45.888653,124.634133,152.252451
1,18,28469.585938,6503.035645,19.043039,8.590194,2415.484863,45.925720,231.634728,178.787583


## Filter data

In [8]:
# Apply filters

# change range of each filters
# comment filters that you do not need

# Select frames by changing the numbers 100 and 1000
df_selected=df_selected[(df_selected['Frame'] >=100) & (df_selected['Frame'] <=1000)] 

# Select photons by changing the numbers 300 and 5000
df_selected=df_selected[(df_selected['Photons'] >=300) & (df_selected['Photons'] <=5000)]

# Select X precision by changing the numbers 5 and 20
df_selected=df_selected[(df_selected['X precision (nm)'] >=5) & (df_selected['X precision (nm)'] <=20)]

# Select Y precision by changing the numbers 5 and 20
df_selected=df_selected[(df_selected['Y precision (nm)'] >=5) & (df_selected['Y precision (nm)'] <=20)]

# Select background less than 50
df_selected=df_selected[df_selected['Background'] < 50]

# Similarly, filters to other columns can be applied

In [9]:
# Look at the top 5 rows of the selected data
df_selected.head()

,Frame,X (nm),Y (nm),X precision (nm),Y precision (nm),Photons,Background,PSF Sigma X (nm),PSF Sigma Y (nm)
3408,100,38154.527344,4343.988770,18.230947,15.368068,3348.474854,17.822321,341.204760,329.559633
3413,100,2763.216553,7205.334473,11.824074,14.438939,413.026886,41.141224,98.622108,128.444472
3428,100,9495.045898,33588.937500,9.045322,9.548347,2478.431152,47.241631,215.029269,225.304560
3429,100,1995.716675,35494.394531,17.268505,16.068125,609.832336,46.932186,166.893714,154.006983
3434,100,42532.035156,47797.285156,15.417689,19.281782,547.563354,48.882065,137.058246,172.986957


In [10]:
# Look at the number of rows that passed the filter.
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 3408 to 39045
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Frame             4521 non-null   int64  
 1   X (nm)            4521 non-null   float64
 2   Y (nm)            4521 non-null   float64
 3   X precision (nm)  4521 non-null   float64
 4   Y precision (nm)  4521 non-null   float64
 5   Photons           4521 non-null   float64
 6   Background        4521 non-null   float64
 7   PSF Sigma X (nm)  4521 non-null   float64
 8   PSF Sigma Y (nm)  4521 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 353.2 KB


## Export selected data as csv

In [11]:
# Write the localisation file csv
df_selected.to_csv('Nanoimager_localizations_filtered.csv', index=False)